In [1]:
import json
import numpy as np
import pandas as pd
from sklearn import preprocessing
import Representation_learning
from sklearn.neighbors import LocalOutlierFactor as LOF
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
# from xgboost.sklearn import XGBClassifier
# import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from models import Net, DQN

OSError: /home/prudvi/project/env/lib/python3.8/site-packages/torch/lib/../../nvidia/cublas/lib/libcublas.so.11: symbol cublasLtGetStatusString version libcublasLt.so.11 not defined in file libcublasLt.so.11 with link time reference

In [2]:
print('Loading data ...')
dataset = pd.read_csv('data/breast_cancer_transcript_expression_with_label.csv')
print(f'Loaded {dataset.shape = }')
dataset = dataset.set_index('Unnamed: 0').T
print(f'Transpose {dataset.shape = }')
print('Data loaded.')

Loading data ...


/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,2

Loaded dataset.shape = (93651, 1049)
Transpose dataset.shape = (1048, 93651)
Data loaded.


In [3]:
r, c = dataset.shape
array = dataset.values
print(f'{dataset.shape = }')
# Y = dataset.iloc[:,0]
# X = dataset.iloc[:,1:c]
X = dataset.iloc[:,0:(c-1)]
Y = dataset.iloc[:,(c-1)]
print(f'{X.shape = }, {Y.shape = }')
io = open("IGorderbreastcancer.txt","r")
order = np.array(json.load(io), dtype=np.int32)

print(f'{order.shape = }')
X = X.iloc[:, order]
print(f'{X.shape = }')

dataset.shape = (1048, 93651)
X.shape = (1048, 93650), Y.shape = (1048,)
order.shape = (93650,)
X.shape = (1048, 93650)


In [4]:
X_train, X_val, Y_train, Y_val = model_selection.train_test_split(X, Y, test_size=0.1, random_state=0)
print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)
print('train_test_split done ...')

print('Random forest and DQN initialising ...')
model = RandomForestClassifier(n_jobs=-1,n_estimators=100, random_state=0)
#model =DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3, random_state=0)

N_feature = X_train.shape[1] # feature number
N_sample = X_train.shape[0] # feature length,i.e., sample number

BATCH_SIZE = 32
LR = 0.01
#EPSILON = 0.9
GAMMA = 0.9
TARGET_REPLACE_ITER = 100 # After how much time you refresh target network
MEMORY_CAPACITY = 400 # The size of experience replay buffer
EXPLORE_STEPS = 1000# How many exploration steps you'd like, should be larger than MEMORY_CAPACITY20
N_ACTIONS = 2
N_STATES = 480+N_feature

np.random.seed(0)
torch.manual_seed(0)    # reproducible

dqn = DQN(N_STATES=N_STATES, N_ACTIONS=N_ACTIONS)

Fstate = np.random.randint(2, size=N_feature)
while sum(Fstate) < 2:
    Fstate = np.random.randint(2, size=N_feature)

print('done.')

# Fstate = np.zeros(N_feature)
# Fstate[0] = 1
print('Pre-processing data ...')
X_selected = X_train.iloc[:, Fstate == 1]
X_array = np.array(X_selected)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
X_array = min_max_scaler.fit_transform(X_array)
print('done.')

(943, 93650) (105, 93650) (943,) (105,)
train_test_split done ...
Random forest and DQN initialising ...
==== N_STATES = 94610 ====
==== N_STATES = 94610 ====
done.
Pre-processing data ...
done.


In [5]:
# print('Representation learning ...')
# X_tensor = torch.FloatTensor(X_array).unsqueeze(0).unsqueeze(0)
# s = Representation_learning.representation_training(X_tensor)
# s = s.detach().numpy().reshape(-1)
# position = np.arange(1, N_feature+1)
# print(s.shape, position.shape)
# import pandas as pd 
# pd.DataFrame(s).to_csv("breast_cancer_representation_learning.csv")
# print('done.')


In [6]:
# load representation learning 
df = pd.read_csv("breast_cancer_representation_learning.csv")
s = np.array(df.to_dict('split')['data'])
position = np.arange(1, N_feature+1)
print(s.shape, position.shape)

(480, 2) (93650,)


In [7]:
print('One hot encoding data ...')
onehot_encoded = OneHotEncoder(sparse=False).fit_transform(position.reshape(-1, 1))
s = np.append(s, onehot_encoded[0])  
print(s.shape)  
print('done.')

One hot encoding data ...
(94610,)
done.


In [1]:
result = []
T = N_feature
# dqn.EPSILON_STEP_SIZE = (dqn.EPSILON_MAX - dqn.EPSILON)/((EXPLORE_STEPS-1)*T)
for i in range(EXPLORE_STEPS):
    t = i % T
    Faction = dqn.choose_action(s)
    Fstate[t] = Faction
    if sum(Fstate) < 1:
        Faction = 1
        Fstate[t] = Faction

    X_selected = X_train.iloc[:, Fstate == 1]
    X_array = np.array(X_selected)
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    X_array = min_max_scaler.fit_transform(X_array)
    X_tensor = torch.FloatTensor(X_array).unsqueeze(0).unsqueeze(0)
    s_ = Representation_learning.representation_training(X_tensor)
    s_ = s_.detach().numpy().reshape(-1)
    if t == T-1:
        s_ = np.append(s_, onehot_encoded[0])
        # s_ = np.append(s_, 1)
    else:
        s_ = np.append(s_, onehot_encoded[t + 1])
        # s_ = np.append(s_, t + 2)

    model.fit(X_train.iloc[:, Fstate == 1], Y_train)
    accuracy = model.score(X_val.iloc[:, Fstate == 1], Y_val)
    Y_pred = model.predict(X_val.iloc[:, Fstate == 1])
    macroF1 = f1_score(Y_val, Y_pred, average='macro')
    precision = precision_score(Y_val, Y_pred, average='macro')
    recall = recall_score(Y_val, Y_pred, average='macro')

    corr = X_val.corr().abs()
    ave_corr = (corr.iloc[:, t].sum())/ (X_val.shape[1])

    # ave_corr = X_val.corr().abs().sum().sum() / (X_val.shape[0] * X_val.shape[1])
    r = (accuracy - ave_corr)
    # r = accuracy

    dqn.store_transition(s, Faction, r, s_)

    if dqn.memory_counter > MEMORY_CAPACITY:
        dqn.learn()
    print('{}/{}: {}, {}'.format(i+1, EXPLORE_STEPS, r, accuracy), end='\r')
    s = s_
    result.append([accuracy, precision, recall, macroF1, Fstate])

print('done')    

NameError: name 'N_feature' is not defined

In [ ]:
output =[]
name = []
name.append("result types")
output.append("over_Gly")

max_accuracy = 0
optimal_set = []
for i in range(len(result)):
    name.append("Accuracy of the {}-th explore step".format(i))
    output.append(result[i][0])

    if result[i][0] > max_accuracy:
        max_accuracy = result[i][0]
        optimal_set = result[i][4]
        Mmacro_f1 = result[i][3]
        Mprecision = result[i][1]
        Mrecall = result[i][2]

print("The maximum accuracy is: {}, the optimal selection for each feature is:{}".format(max_accuracy, optimal_set))

In [ ]:
name.append("feature subset")
output.append(optimal_set)
name.append("max_accuracy")
output.append(max_accuracy)
name.append("precision")
output.append(Mprecision)
name.append("recall_RF")
output.append(Mrecall)
name.append("macro_f1")
output.append(Mmacro_f1)


out = dict(zip(name, output))
out = pd.DataFrame([out])
out.to_csv('data/breast_cancer_result_predictor.csv', mode='a')